## ДЗ 4. Панков Викентий

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pysam
from Bio import SeqIO
from collections import Counter

## Метод:
Сравниваем риды с одинаковым ID из исходного и скорректированного файлов  

**Для каждой позиции исходного рида и соответствующей позиции скорректированного рида смотрим:**   
    1. если исходная позиция выровнялась правильно, а скорректированная - неправильно, считаем это Falsely corrected error  
    2. если исходная и скорректированная позиции выровнялись правильно, считаем это Correctly unmodified base  
    3. если исходная позиция выровнялась неправильно, а соответствующая скорректированная стала правильной - это Detected & corrected error  
    4. если и исходная, и скорректированная позиции выровнялись неправильно - это Undetected error  
    5. если исходная позиция была правильной, а скорректированная удалена (заменена буквой N) - это Incorrectly removed base  
    6. если исходная позиция была неправильной, а скорректированная удалена - это Detected and removed error 

*** TODO: Учесть инсерции/делеции ***

In [132]:
def estimateErrors(originalFileName, correctedFileName):

    original_align = pysam.AlignmentFile(originalFileName, "rb" )
    corrected_align = pysam.AlignmentFile(correctedFileName, "rb" )

    mdTags = 0
    count = 0

    undetected = 0
    detectedAndCorrected = 0 
    detectedAndRemoved = 0
    falselyCorrected = 0
    correctlyUnmodified = 0
    incorrectlyRemoved = 0

    for original_read, corrected_read in zip(original_align.fetch(), corrected_align.fetch()):
        try:

            if(original_read.cigartuples is None or original_read.cigartuples is None):
                continue

            origin_query = original_read.query_sequence
            origin_aligns = original_read.get_aligned_pairs(matches_only = True, with_seq = True)
            corrected_query = corrected_read.query_sequence
            corrected_aligns = corrected_read.get_aligned_pairs(matches_only = True, with_seq = True)

            for original_tuple, corrected_tuple in zip(origin_aligns, corrected_aligns):
                origin_base = origin_query[original_tuple[0]] #нуклеотид в исходном риде
                corrected_base = corrected_query[corrected_tuple[0]] #нуклеотид в скорректированном риде


                if(origin_base != original_tuple[2] and corrected_base == "N"):
                    detectedAndRemoved+=1
                elif(origin_base != original_tuple[2] and corrected_base == corrected_tuple[2]): #_tuple[2] - нуклеотид в референсе
                    detectedAndCorrected+=1
                elif(origin_base != original_tuple[2] and corrected_base != corrected_tuple[2]):
                    undetected+=1
                elif(origin_base == original_tuple[2] and corrected_base == "N"):
                    incorrectlyRemoved+=1
                elif(origin_base == original_tuple[2] and corrected_base != corrected_tuple[2]):
                    falselyCorrected+=1
                elif(origin_base == original_tuple[2] and corrected_base == corrected_tuple[2]):
                    correctlyUnmodified+=1
        except ValueError:
            mdTags+=1

    df = pd.DataFrame([["Error in raw data", "", "", ""], ["Correct base in raw data", "", "", ""]], columns = ["-", "Error in corrected reads", "Correct base in corrected reads", "Base is absent in corrected reads"])
    df = df.set_index(df.columns[0])
    df.loc["Error in raw data", "Error in corrected reads"]=undetected
    df.loc["Error in raw data", "Correct base in corrected reads"]=detectedAndCorrected
    df.loc["Error in raw data", "Base is absent in corrected reads"]=detectedAndRemoved

    df.loc["Correct base in raw data", "Error in corrected reads"]=falselyCorrected
    df.loc["Correct base in raw data", "Correct base in corrected reads"]=correctlyUnmodified
    df.loc["Correct base in raw data", "Base is absent in corrected reads"]=incorrectlyRemoved

    return df


## Trimmomatic:

In [133]:
estimateErrors("data/HW4/original2.bam", "data/HW4/trim2.bam")

,Error in corrected reads,Correct base in corrected reads,Base is absent in corrected reads
-,,,
Error in raw data,9662,2132767,1206
Correct base in raw data,910197,239174223,140355


## SPAdes / BayesHammer:

bwa mem reference.fasta bayeshammer/corrected/ecoli_400K_err_1.00.0_0.cor.fastq.gz bayeshammer/corrected/ecoli_400K_err_2.00.0_0.cor.fastq.gz | samtools sort -o bayeshammer.bam

In [134]:
estimateErrors("data/HW4/original2.bam", "data/HW4/bayeshammer.bam")

,Error in corrected reads,Correct base in corrected reads,Base is absent in corrected reads
-,,,
Error in raw data,2190,2166270,6263
Correct base in raw data,210186,242597992,406992
